In [52]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
import pandas as pd
import numpy as np

import inventario_finanzas_int.funciones_auxiliares as fa

pd.set_option("display.max_columns", None)

In [54]:
df_mobiliario_consolidado = pd.read_excel(
    "../data/raw/MOBILIARIO (ARIEL)/MATRIZ CORRELATIVA AÑO 2025.xlsx"
)

In [55]:
df_mobiliario = pd.read_csv("../data/processed/df_procesada_mobiliarios.csv")

# Lee equipos medicos y los ordena
df_equipos_medicos = pd.read_csv("../data/processed/df_procesada_equipos_medicos.csv")
df_equipos_medicos = df_equipos_medicos.sort_values(
    ["piso", "unidadservicio_clinico", "ubicacion_unidad", "bien"],
    ascending=[True, True, False, True],
)

# lee Industriales, y elimina las columnas completamente vacias
df_industriales = pd.read_csv("../data/processed/df_procesada_industriales.csv")
df_industriales = df_industriales.dropna(axis=1, how="all")
df_industriales = df_industriales.sort_values(
    ["piso", "unidadservicio_clinico", "ubicacion_unidad", "bien"],
    ascending=[True, True, False, True],
)

# Lee Informaticos
df_informaticos = pd.read_csv("../data/processed/df_procesada_informaticos.csv")

In [56]:
union_bienes_con_informatico = pd.concat(
    [df_mobiliario, df_equipos_medicos, df_industriales, df_informaticos]
)
union_bienes_con_informatico["conteo"] = 1

union_bienes_sin_informatico = pd.concat([df_mobiliario, df_equipos_medicos, df_industriales])
union_bienes_sin_informatico["conteo"] = 1

In [57]:
conteo_equipos_con_informatico = pd.pivot_table(
    union_bienes_con_informatico,
    columns="tipo_bien",
    index="unidadservicio_clinico",
    values="conteo",
    fill_value=0,
    aggfunc="sum",
)

conteo_equipos_sin_informatico = pd.pivot_table(
    union_bienes_sin_informatico,
    columns="tipo_bien",
    index="unidadservicio_clinico",
    values="conteo",
    fill_value=0,
    aggfunc="sum",
)

## Asignacion de Correlativos

Se hara de forma separada

In [58]:
GLOSAS_EQUIPOS_MEDICOS_CON_CORRELATIVO = ["INT", "INT (SSMO)", "INT (MINSAL)", "COMODATO"]
GLOSAS_INDUSTRIALES_CON_CORRELATIVO = ["INT", "INT (MEL)", "INT (DONACION)", "COMODATO"]

In [59]:
ULTIMO_CORRELATIVO = 4015 + 1
# Identifica los registros a los que hay que asignar correlativo
mask_correlativo_medicos = df_equipos_medicos["propiedad"].isin(
    GLOSAS_EQUIPOS_MEDICOS_CON_CORRELATIVO
)
df_equipos_medicos_con_correlativos = df_equipos_medicos[mask_correlativo_medicos]

# Asigna correlativos a los registros que requieren correlativo
correlativos_equipos_medicos = [
    f"2025-{i}"
    for i in range(
        ULTIMO_CORRELATIVO, ULTIMO_CORRELATIVO + len(df_equipos_medicos_con_correlativos)
    )
]
df_equipos_medicos.loc[mask_correlativo_medicos, "correlativo_2025"] = correlativos_equipos_medicos
df_equipos_medicos["correlativo_2025"] = df_equipos_medicos["correlativo_2025"].replace("nan", np.nan)

In [60]:
df_equipos_medicos

,correlativo_antiguo,bien,marca,modelo,serie,unidadservicio_clinico,ubicacion_unidad,piso,propiedad,unidad_hoja,tipo_bien,correlativo_2025
1,SIN INVENTARIO,ANDADOR CON RUEDAS,BLUDING,AD-03,NaN,MEDICINA FISICA Y REHABILITACION,GIMNASIO,-1.0,INT,MQ 3 NORTE,EQUIPO MEDICO,2025-4016
0,3-6555,CICLOERGOMETRO,MONARK,ERGOMETRICA,9035082,MEDICINA FISICA Y REHABILITACION,GIMNASIO,-1.0,INT,MQ 3 NORTE,EQUIPO MEDICO,2025-4017
71,INTEM-SS-A3370,PROGRAMADOR MARCAPASOS,MEDTRONIC,NaN,PKK055746R,CONSULTORIO EXTERNO,SALA MARCAPASOS,1.0,INT,CONSUL. EXTERNO,EQUIPO MEDICO,2025-4018
66,(3-9733),COMPUTADOR DE REGISTROS HOLTER,MORTARA,HSCROBE5,NaN,CONSULTORIO EXTERNO,SALA ENFERMERIA,1.0,INT,CONSUL. EXTERNO,EQUIPO MEDICO,2025-4019
67,1499-6060,GRABADOR DE HOLTER DE ARRITMIA,BAXTER MORTARA,H3+,NaN,CONSULTORIO EXTERNO,SALA ENFERMERIA,1.0,ARRIENDO,CONSUL. EXTERNO,EQUIPO MEDICO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
831,INTEM-SS-A3408,VENTILADOR MECANICO NO INVASIVO,PHILIPS,V60,NaN,UPC 5 NORTE,BODEGA EQUIPOS,5.0,INT,UPC 5 NORTE,EQUIPO MEDICO,2025-4975
832,INTEM-SS-A3415,VENTILADOR MECANICO NO INVASIVO,PHILIPS,V60,NaN,UPC 5 NORTE,BODEGA EQUIPOS,5.0,INT,UPC 5 NORTE,EQUIPO MEDICO,2025-4976
833,SIN INVENTARIO,VENTILADOR MECANICO NO INVASIVO,PURITAN BENNETT,NaN,40966N1131,UPC 5 NORTE,BODEGA EQUIPOS,5.0,INT,UPC 5 NORTE,EQUIPO MEDICO,2025-4977
972,1499-4556,ASPIRADOR ELECTRONIC,ACCUMAX,PIPET CONTROLLER,SE935956,LABORATORIO,LABORATORIO DE TUBERCULOSIS,NaN,INT,UMT Y LAB.,EQUIPO MEDICO,2025-4978


In [61]:
# Aisla el ultimo correlativo asignado
ULTIMO_CORRELATIVO = 4979 + 1

# Identifica los registros a los que hay que asignar correlativo
mask_correlativos_ind = df_industriales["propiedad"].isin(GLOSAS_INDUSTRIALES_CON_CORRELATIVO)
df_industriales_con_correlativos = df_industriales[mask_correlativos_ind]

# Asigna los correlativos a los registros que requieren correlativo
correlativos_industriales = [
    f"2025-{i}"
    for i in range(ULTIMO_CORRELATIVO, ULTIMO_CORRELATIVO + len(df_industriales_con_correlativos))
]

df_industriales.loc[mask_correlativos_ind, "correlativo_2025"] = correlativos_industriales
df_industriales["correlativo_2025"] = df_industriales["correlativo_2025"].replace("nan", np.nan)

In [62]:
# Genera la matriz total de bienes
matriz_total_correlativos = pd.concat([df_equipos_medicos, df_industriales])

In [63]:
# Exporta asignando correlativos
df_equipos_medicos.to_excel("../data/interim/matriz_equipos_medicos_2025.xlsx", index=False)
df_industriales.to_excel("../data/interim/matriz_industriales_2025.xlsx", index=False)
matriz_total_correlativos.to_excel("../data/interim/matriz_bienes_INT_2025.xlsx", index=False)